# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [1]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

Loading required package: usethis

Loading required package: reshape


Attaching package: ‘rapportools’


The following objects are masked from ‘package:stats’:

    IQR, median, sd, var


The following objects are masked from ‘package:base’:

    max, mean, min, range, sum



Attaching package: ‘data.table’


The following object is masked from ‘package:reshape’:

    melt


Loading required package: tfruns



In [2]:
# create a CubeAPI object
cube_api = CubeAPI$new()

INFO [2020-12-31 18:35:26] Version: 0.0.2
INFO [2020-12-31 18:35:26] https://metadata-service-sqa.azurewebsites.net/


In [3]:
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

INFO [2020-12-31 18:35:30] verification_uri_complete: https://thejacksonlaboratory.auth0.com/activate?user_code=XVFR-SHSD


### Get Metadata Definition

In [4]:
# retrieve metadata element
response = cube_api$get_element()

INFO [2020-12-31 18:35:38] GET: https://metadata-service-sqa.azurewebsites.net/metadata_definition/element
INFO [2020-12-31 18:35:38] status_code: 403


No encoding supplied: defaulting to UTF-8.



ERROR [2020-12-31 18:35:39] response: <!DOCTYPE html>
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>Web App - Unavailable</title>
<style type="text/css">
        html {
            height: 100%;
            width: 100%;
        }

        #feature {
            width: 960px;
            margin: 95px auto 0 auto;
            overflow: auto;
        }

        #content {
            font-family: "Segoe UI";
            font-weight: normal;
            font-size: 22px;
            color: #ffffff;
            float: left;
            width: 460px;
            margin-top: 68px;
            margin-left: 0px;
            vertical-align: middle;
        }

            #content h1 {
                font-family: "Segoe UI Light";
                color: #ffffff;
                font-weight: normal;
                font-size: 60px;
                line-height: 48pt;
                width: 800px;
            }

        p a, p a:visited, p a:active, p a:hov

In [ ]:
str(response)

In [ ]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [ ]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

### Get Bucket data from metadata collection

In [ ]:
storage_info = cube_api$get_metadata_collection_storage_info()

In [ ]:
storage_info

In [ ]:
## get bucket data
bucket_data = get_bucket_data(bucket_name = storage_info[2, 3], file_name = storage_info[2, 4] )
bucket_data

<br/><br/>

### Get Metadata Definition with a Parameter

In [ ]:
response = cube_api$get_element(element_id = 85)

In [ ]:
str(content(response))

### Get Metadata with accession ids 

In [ ]:
response = cube_api$get_element_instance(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

### Get Metadata Collection

In [ ]:
response = cube_api$get_metadata_collection()

In [ ]:
json = response_json_to_data(response)

In [ ]:
str(json$results)

In [ ]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

In [ ]:
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

In [ ]:
results = content(response)$results
results[[4]][[4]]

In [ ]:
data_url = results[[4]][[4]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
df

### Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_api$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_api$get_end_point("/metadata_repository/collection_item/1/")

In [ ]:
content(response)

### Post endpoint

In [ ]:
response = cube_api$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### More Information

In [ ]:
?CubeAPI

In [ ]:
?Cube